In [2]:
# Convert .pts and .seg to .txt
import os

def convert_files_to_txt(folder, old_ext, new_ext=".txt"):
    """
    Converts all files in the given folder from one extension to another.
    """
    for filename in os.listdir(folder):
        if filename.endswith(old_ext):
            old_path = os.path.join(folder, filename)
            new_filename = filename.replace(old_ext, new_ext)
            new_path = os.path.join(folder, new_filename)

            # Read the file as text and write to new file
            with open(old_path, "r") as f:
                content = f.readlines()
            with open(new_path, "w") as f:
                f.writelines(content)

            print(f"Converted: {old_path} -> {new_path}")

# Define the folders
points_folder = "data/points"
labels_folder = "data/labels"

# Convert .pts to .txt
convert_files_to_txt(points_folder, ".pts")

# Convert .seg to .txt
convert_files_to_txt(labels_folder, ".seg")

Converted: data/points/157016.pts -> data/points/157016.txt
Converted: data/points/813200.pts -> data/points/813200.txt
Converted: data/points/382240.pts -> data/points/382240.txt
Converted: data/points/592036.pts -> data/points/592036.txt
Converted: data/points/807665.pts -> data/points/807665.txt
Converted: data/points/199395.pts -> data/points/199395.txt
Converted: data/points/740334.pts -> data/points/740334.txt
Converted: data/points/698039.pts -> data/points/698039.txt
Converted: data/points/811402.pts -> data/points/811402.txt
Converted: data/points/825957.pts -> data/points/825957.txt
Converted: data/points/889317.pts -> data/points/889317.txt
Converted: data/points/79349.pts -> data/points/79349.txt
Converted: data/points/625143.pts -> data/points/625143.txt
Converted: data/points/1014817.pts -> data/points/1014817.txt
Converted: data/points/202723.pts -> data/points/202723.txt
Converted: data/points/1063647.pts -> data/points/1063647.txt
Converted: data/points/326330.pts -> d

In [3]:
def find_max_pointcloud_length(folder, extension=".txt"):
    """
    Finds the maximum number of points in all point cloud files.
    """
    max_length = 0
    for filename in os.listdir(folder):
        if filename.endswith(extension):
            file_path = os.path.join(folder, filename)
            with open(file_path, "r") as f:
                num_lines = sum(1 for _ in f)  # Count lines (points)
            max_length = max(max_length, num_lines)
            print(f"{filename}: {num_lines} points")

    print(f"Maximum point cloud length: {max_length}")
    return max_length

# Find max length in point cloud files
max_points = find_max_pointcloud_length(points_folder)

552907.txt: 571 points
59384.txt: 147 points
724626.txt: 422 points
708643.txt: 422 points
571310.txt: 228 points
471304.txt: 118 points
878756.txt: 313 points
361210.txt: 264 points
36384.txt: 375 points
823632.txt: 363 points
632454.txt: 198 points
802818.txt: 215 points
920677.txt: 269 points
903078.txt: 362 points
495646.txt: 285 points
505269.txt: 638 points
243701.txt: 190 points
1062914.txt: 398 points
572181.txt: 352 points
857207.txt: 225 points
1054136.txt: 1252 points
957577.txt: 179 points
994104.txt: 185 points
543564.txt: 155 points
56103.txt: 145 points
727502.txt: 136 points
663619.txt: 342 points
378031.txt: 452 points
200029.txt: 313 points
846490.txt: 247 points
478875.txt: 220 points
520398.txt: 287 points
298089.txt: 330 points
167998.txt: 226 points
248860.txt: 186 points
775719.txt: 201 points
1027254.txt: 226 points
894459.txt: 368 points
530799.txt: 344 points
883680.txt: 267 points
115738.txt: 605 points
354086.txt: 152 points
243288.txt: 167 points
423041.txt

In [1]:
import os
import random
import shutil

def count_files_in_folder(folder, extension=".txt"):
    """
    Counts the number of files in the given folder with the given extension.
    """
    num_files = 0
    for filename in os.listdir(folder):
        if filename.endswith(extension):
            num_files += 1
    print(f"Number of {extension} files in {folder}: {num_files}")
    return num_files

def pick_random_related_files(points_folder, labels_folder, sample_points_folder, sample_labels_folder, num_files=100, extension=".txt"):
    """
    Picks random related files from the points and labels folders and copies them to the sample folders.
    """
    if not os.path.exists(sample_points_folder):
        os.makedirs(sample_points_folder)
    if not os.path.exists(sample_labels_folder):
        os.makedirs(sample_labels_folder)
    
    points_files = [f for f in os.listdir(points_folder) if f.endswith(extension)]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith(extension)]
    
    # Find common files in both folders
    common_files = list(set(points_files) & set(labels_files))
    
    selected_files = random.sample(common_files, min(num_files, len(common_files)))
    
    for file in selected_files:
        shutil.copy(os.path.join(points_folder, file), os.path.join(sample_points_folder, file))
        shutil.copy(os.path.join(labels_folder, file), os.path.join(sample_labels_folder, file))
    
    print(f"Copied {len(selected_files)} related files to {sample_points_folder} and {sample_labels_folder}")

# Define source and destination folders
points_folder = "data/points"
labels_folder = "data/labels"
sample_points_folder = "data/sample_points_50"
sample_labels_folder = "data/sample_labels_50"

# Count files in the folders
num_points_files = count_files_in_folder(points_folder)
num_labels_files = count_files_in_folder(labels_folder)

# Pick random 100 related files from both folders
pick_random_related_files(points_folder, labels_folder, sample_points_folder, sample_labels_folder, num_files=50, extension=".txt")

Number of .txt files in data/points: 219
Number of .txt files in data/labels: 219
Copied 50 related files to data/sample_points_50 and data/sample_labels_50
